# Subconsultas, Tabelas Derivadas e Expressões Comuns de Tabela

Nesta seção, aprenderemos alguns padrões mais avançados para incorporar consultas dentro de consultas. Isso pode criar algumas ferramentas úteis para análise de dados, incluindo a substituição de valores ausentes.

Vamos começar com o básico sobre subconsultas e, em seguida, avançar para tabelas derivadas e expressões comuns de tabela.

Conecte-se ao banco de dados `company_operations.db` e vamos nos concentrar primeiro na tabela `WEATHER_MONITOR`.

In [ ]:
import sqlite3
import pandas as pd 

conn = sqlite3.connect('company_operations.db')

pd.read_sql("SELECT * FROM WEATHER_MONITOR LIMIT 10", conn)

## Subconsultas

Observe como a coluna `TEMPERATURE` apresenta valores nulos ausentes. Não há muitos. Apenas 3 registros.

In [ ]:
sql = """
SELECT * FROM WEATHER_MONITOR
WHERE TEMPERATURE IS NULL 
"""
            
pd.read_sql(sql, conn)


Digamos que queremos substituir esses valores nulos pela média de `TEMPERATURE` em todos os registros. Talvez façamos isso para não descartar os três registros e ainda usá-los para modelagem. Podemos fazer isso usando uma **subconsulta escalar** como esta, que incorpora uma consulta retornando um único valor dentro de uma consulta.

In [ ]:
sql = """
SELECT ID, 
REPORT_CODE, 
REPORT_DATE, 
LOCATION_ID, 

CASE WHEN TEMPERATURE IS NULL THEN (SELECT AVG(TEMPERATURE) FROM WEATHER_MONITOR) 
     ELSE TEMPERATURE 
END AS TEMPERATURE_IMPUTED

FROM WEATHER_MONITOR
WHERE TEMPERATURE IS NULL 
"""
            
pd.read_sql(sql, conn)


Talvez seja mais preciso considerar o mês e o ano e extrair médias apenas dos campos que correspondem a esses dois atributos. Afinal, observe como a temperatura média varia de acordo com `YEAR` e `MONTH`. Observe que não há campos `YEAR` e `MONTH`, pois podemos inferir isso do campo `REPORT_DATE`. No SQLite, a maneira como fazemos isso é usar `strftime()` com uma [sintaxe de padrão especial](https://www.sqlite.org/lang_datefunc.html) para converter elementos de data e hora. Usamos `%Y` para o ano e `%m` para o mês.

In [ ]:
sql = """ 
SELECT strftime('%Y', REPORT_DATE) AS YEAR, 
strftime('%m', REPORT_DATE) AS MONTH,
AVG(TEMPERATURE) AS AVG_TEMP 
FROM WEATHER_MONITOR
GROUP BY 1, 2
"""

pd.read_sql(sql, conn)

Isso é um pouco complicado. Em nossa subconsulta correlacionada, trabalharemos com duas instâncias da tabela `WEATHER_MONITOR`, onde usaremos o alias `wm2` para a instância da subconsulta e `wm1` para a instância externa. Podemos então garantir que a subconsulta seja iniciada para todos os registros, consultando apenas os registros correspondentes ao ano e ao mês.

In [ ]:
sql = """
SELECT ID, 
REPORT_CODE, 
REPORT_DATE, 
LOCATION_ID, 


CASE WHEN TEMPERATURE IS NULL THEN 
    (
        SELECT AVG(TEMPERATURE) FROM WEATHER_MONITOR wm2 
        WHERE strftime('%Y', wm1.REPORT_DATE) = strftime('%Y', wm2.REPORT_DATE) -- year must match outer record
        AND strftime('%m', wm1.REPORT_DATE) = strftime('%m', wm2.REPORT_DATE) -- month must match outer record
     ) 
     ELSE TEMPERATURE 
END AS TEMPERATURE_IMPUTED

FROM WEATHER_MONITOR wm1 
WHERE TEMPERATURE IS NULL 
"""
            
pd.read_sql(sql, conn)


Existem maneiras mais eficientes de fazer isso, incluindo tabelas derivadas e expressões de tabela comuns. Abordaremos essas informações em breve.

Vamos nos concentrar em duas outras tabelas, `CUSTOMER` e `CUSTOMER_ORDER`.

In [ ]:
pd.read_sql("SELECT * FROM CUSTOMER", conn)

In [ ]:
pd.read_sql("SELECT * FROM CUSTOMER_ORDER", conn)

Essas duas tabelas são vinculadas pelo `CUSTOMER_ID`, o que significa que cada registro `CUSTOMER_ORDER` possui um `CUSTOMER_ID` associado a ele. Podemos então usar esse valor `CUSTOMER_ID` para consultar os detalhes de `CUSTOMER`.

O que pretendemos alcançar aqui é encontrar registros `CUSTOMER` que não tenham registros `CUSTOMER_ORDER` associados a eles. A maneira mais rudimentar de fazer isso é com um operador `LEFT JOIN`, que incluirá todos os registros na tabela "esquerda", mesmo que não haja registros para unir na tabela "direita". "Esquerda" e "direita" determinam por qual lado uma tabela é especificada em relação às palavras-chave do operador `LEFT JOIN`. Se não houver registros na tabela da direita, esses campos da tabela da direita serão `NULL` em um registro de espaço reservado. Podemos verificar se esses campos na tabela da direita são nulos como resultado do `LEFT JOIN`. Esta é a técnica para encontrar clientes que não têm pedidos.

In [ ]:
sql = """
SELECT CUSTOMER.CUSTOMER_ID, 
CUSTOMER_NAME

FROM CUSTOMER LEFT JOIN CUSTOMER_ORDER
ON CUSTOMER.CUSTOMER_ID = CUSTOMER_ORDER.CUSTOMER_ID

WHERE CUSTOMER_ORDER.CUSTOMER_ID IS NULL
"""
            
pd.read_sql(sql, conn)


Uma maneira possivelmente mais elegante de fazer isso é usar subconsultas. Podemos obter um conjunto de valores `CUSTOMER_ID` em uma subconsulta e verificar se há clientes que não tenham um `CUSTOMER_ID` nesses registros `CUSTOMER_ORDER`.

In [ ]:
sql = """
SELECT * FROM CUSTOMER 
WHERE CUSTOMER_ID NOT IN (SELECT DISTINCT CUSTOMER_ID FROM CUSTOMER_ORDER)
"""

pd.read_sql(sql, conn)

Outra maneira de fazer isso é usar o operador `EXISTS` (ou `NOT EXISTS`) para encontrar quaisquer registros `CUSTOMER` existentes que atendam à condição `WHERE`, procurando por registros `CUSTOMER_ORDER` com esse `CUSTOMER_ID`. Podemos aproveitar o fato de que ele não fará uma varredura completa da tabela, mas parará no momento em que encontrar um único registro.

In [ ]:
sql = """
SELECT * FROM CUSTOMER c 
WHERE NOT EXISTS (SELECT * FROM CUSTOMER_ORDER WHERE CUSTOMER_ID = c.CUSTOMER_ID)
"""

pd.read_sql(sql, conn)

Permita-me incluir apenas mais um exemplo. Também podemos usar subconsultas para retornar apenas pedidos da `ORDER_DATE` mais recente.

In [ ]:
sql = """
SELECT * FROM CUSTOMER_ORDER
WHERE ORDER_DATE = (SELECT MAX(ORDER_DATE) FROM CUSTOMER_ORDER)
"""

pd.read_sql(sql, conn)

## Tabelas Derivadas

Lembre-se de que demonstramos esta consulta mostrando a temperatura média por ano e mês.

In [ ]:
sql = """ 
SELECT strftime('%Y', REPORT_DATE) AS YEAR, 
strftime('%m', REPORT_DATE) AS MONTH,
AVG(TEMPERATURE) AS AVG_TEMP 
FROM WEATHER_MONITOR
GROUP BY 1, 2
"""

pd.read_sql(sql, conn)

E se juntássemos esta "tabela" (apoiada por uma consulta `SELECT`) a `WEATHER_MONITOR` e imputássemos os três valores ausentes de `TEMPERATURE` com as médias daquele ano e mês? Consultar outra consulta `SELECT` dessa forma e tratá-la como uma tabela é conhecido como uma **tabela derivada**. Observe abaixo que incorporamos essa consulta `SELECT` ao `INNER JOIN` e a tratamos como uma tabela, unindo-a com base no ano e mês.

In [ ]:
sql = """
SELECT ID, 
REPORT_CODE, 
REPORT_DATE, 
LOCATION_ID, 
CASE WHEN TEMPERATURE IS NULL THEN AVG_TEMP ELSE TEMPERATURE END AS TEMPERATURE_IMPUTED

FROM WEATHER_MONITOR INNER JOIN ( 
    SELECT strftime('%Y', REPORT_DATE) AS YEAR, 
    strftime('%m', REPORT_DATE) AS MONTH,
    AVG(TEMPERATURE) AS AVG_TEMP 
    FROM WEATHER_MONITOR
    GROUP BY 1, 2
) AS temp_avgs

ON strftime('%Y', REPORT_DATE) = temp_avgs.YEAR
AND strftime('%m', REPORT_DATE) = temp_avgs.MONTH

WHERE TEMPERATURE IS NULL 
"""
            
pd.read_sql(sql, conn)


Há uma vantagem aqui: calculamos essas médias antecipadamente e, em seguida, as consultamos nos resultados da consulta `SELECT` final. No entanto, tabelas derivadas podem ser aninhadas em várias camadas, criando um antipadrão chamado de **pirâmide da perdição**. Podemos aninhar várias tabelas derivadas em uma consulta, mas isso se torna confuso e difícil de filtrar e gerenciar. Por esse motivo, o desenvolvedor SQL moderno deve optar por usar expressões de tabela comuns.

## Expressões de Tabela Comuns (CTEs)

**Expressões de Tabela Comuns (CTEs)** são suas melhores amigas como desenvolvedor e analista de SQL. Elas dividirão consultas complexas em etapas fáceis de entender. Aqui está nosso exemplo anterior imputando médias para valores ausentes para `TEMPERATURE` em nossa tabela `WEATHER_MONITOR`, mas usando expressões de tabela comuns.

In [ ]:
sql = """
WITH temp_avgs AS (
    SELECT strftime('%Y', REPORT_DATE) AS YEAR, 
    strftime('%m', REPORT_DATE) AS MONTH,
    AVG(TEMPERATURE) AS AVG_TEMP 
    FROM WEATHER_MONITOR
    GROUP BY 1, 2
)

SELECT ID, 
REPORT_CODE, 
REPORT_DATE, 
LOCATION_ID, 
CASE WHEN TEMPERATURE IS NULL THEN AVG_TEMP ELSE TEMPERATURE END AS TEMPERATURE_IMPUTED

FROM WEATHER_MONITOR INNER JOIN temp_avgs

ON strftime('%Y', REPORT_DATE) = temp_avgs.YEAR
AND strftime('%m', REPORT_DATE) = temp_avgs.MONTH

WHERE TEMPERATURE IS NULL 
"""
            
pd.read_sql(sql, conn)


Observe como `temp_avgs` pode ser declarado antecipadamente, tratando-o como uma "tabela" chamada `temp_avgs` apoiada por uma consulta `SELECT`. Isso é muito mais limpo do que incorporá-lo ao corpo da consulta `SELECT` principal.

O que é ainda melhor sobre expressões de tabela comuns é que uma CTE pode se referir a uma CTE anterior, criando uma "cadeia" de etapas que quebra a lógica complexa sem criar pirâmides da perdição. Abaixo, criamos uma CTE imputando as temperaturas ausentes e, em seguida, verificando se as médias substituíram esses valores nulos na consulta final.

In [ ]:
sql = """
WITH temp_avgs AS (
    SELECT strftime('%Y', REPORT_DATE) AS YEAR, 
    strftime('%m', REPORT_DATE) AS MONTH,
    AVG(TEMPERATURE) AS AVG_TEMP 
    FROM WEATHER_MONITOR
    GROUP BY 1, 2
) , 

missing_temps_imputed AS ( 
    SELECT ID, 
    REPORT_CODE, 
    REPORT_DATE, 
    LOCATION_ID, 
    CASE WHEN TEMPERATURE IS NULL THEN temp_avgs.AVG_TEMP ELSE TEMPERATURE END AS TEMPERATURE_IMPUTED
    
    FROM WEATHER_MONITOR INNER JOIN temp_avgs
    
    ON strftime('%Y', REPORT_DATE) = temp_avgs.YEAR 
    AND strftime('%m', REPORT_DATE)  = temp_avgs.MONTH
)

SELECT * FROM missing_temps_imputed
WHERE ID IN (SELECT ID FROM WEATHER_MONITOR WHERE TEMPERATURE IS NULL)
"""
            
pd.read_sql(sql, conn)


## Exercício

Reescreva a consulta abaixo, que mostra a precipitação total para cada mês/ano, juntamente com cada registro `WEATHER_MONITOR`, mas como uma expressão de tabela comum, em vez de uma subconsulta correlacionada. Use o tempo que precisar e rotule as informações como quiser.

In [ ]:
sql = """
SELECT ID, 
REPORT_CODE, 
REPORT_DATE, 
LOCATION_ID, 
RAIN,
(
    SELECT SUM(RAIN) FROM WEATHER_MONITOR wm2 
    WHERE strftime('%Y', wm1.REPORT_DATE) = strftime('%Y', wm2.REPORT_DATE) -- year must match outer record
    AND strftime('%m', wm1.REPORT_DATE) = strftime('%m', wm2.REPORT_DATE) -- month must match outer record
) AS RAIN_TOTAL_MONTH

FROM WEATHER_MONITOR wm1 
"""
            
pd.read_sql(sql, conn)


### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
sql = """ 
WITH rain_totals AS ( 
    SELECT strftime('%Y', REPORT_DATE) AS REPORT_YEAR, 
    strftime('%m', REPORT_DATE) AS REPORT_MONTH, 
    SUM(RAIN) AS TOTAL_RAIN 
    FROM WEATHER_MONITOR  
    GROUP BY 1, 2 
)

SELECT ID, 
REPORT_CODE, 
REPORT_DATE, 
LOCATION_ID, 
RAIN,
TOTAL_RAIN 

FROM WEATHER_MONITOR INNER JOIN rain_totals
ON strftime('%Y', REPORT_DATE) = rain_totals.REPORT_YEAR
AND strftime('%m', REPORT_DATE) = rain_totals.REPORT_MONTH
"""
            
pd.read_sql(sql, conn)
